In [13]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cvxopt as opt
from cvxopt import blas, solvers
import pandas as pd
import zipline



# read one stock from excel and create dataframe 
def read_clean_stock_from_excel(stock):
    # reading from excel 
    df = pd.read_excel(str(stock) + '.xlsx', sheetname=str(stock))

    # dropping unwanted columns and renaming to english 
    df.drop(['日期','成交股數', '成交金額', '最高價', '開盤價', '最低價', '漲跌價差', '成交筆數'], axis=1, inplace=True)
    df.columns=[str(stock)]
    
    return df
    

# concatenate 5 stocks into matrix of daily returns
def concatenate_process_5_stocks(stock_list):
    # add dataframes to df_list
    df_list = []
    for stock in stock_list:
        df_list.append(read_clean_stock_from_excel(stock))   

    # concat df of different stocks 
    result = pd.concat(df_list, axis=1)
    # calculate daily and annual returns of the stocks
    returns_daily = result.pct_change() # Percentage change between the current and a prior element.
    returns_daily = returns_daily.iloc[1:] # Deleting first row since it is NaN, no percentage difference
    returns_matrix = returns_daily.as_matrix().T
    
    return returns_matrix
    
# Seed the generator (start number, multiply by large number, mod of product)
np.random.seed(123)

# Turn off progress printing 
# Part of cvxopt
solvers.options['show_progress'] = False
    
# MAXIMUM 14 stocks 
stock_list = [1101, 1102, 1216, 1301, 1303, 1326, 1402, 2002, 2105,
          2227, 2301, 2303, 2308, 2317, 2327, 2330, 2352, 2354, 2357, 2382, 2395, 2408,
          2409, 2412, 2454, 2474, 2609, 2610, 2633, 2801, 2823, 2880, 2881, 2882, 2883,
          2884, 2885, 2886, 2887, 2890, 2891, 2892, 2912, 3008, 3045, 3481, 4904, 4938, 5871,
          5880, 6505, 9904]
return_vec = concatenate_process_5_stocks(stock_list)

    
    
# Generate random weights that sum to 1
def rand_weights(n):
    ''' Produces n random weights that sum to 1 '''
    k = np.random.rand(n)
    return k / sum(k)



# Evaluate how many of these random portfolios would perform 
# Calculating mean returns and volatility (using SD here)
def random_portfolio(returns):
    ''' 
    Returns the mean and standard deviation of returns for a random portfolio
    '''
    
    # asmatrix Parameter: input data, output data type
    # mean of returns, axis = 1 = column
    p = np.asmatrix(np.mean(returns, axis=1))
    
    # shape returns tuple (n,m), so 0 is number of rows 
    w = np.asmatrix(rand_weights(returns.shape[0]))
    
    # cov: Estimate a covariance matrix, given data and weights
    C = np.asmatrix(np.cov(returns))
    
    # mu = expected return
    # (p = N * 1 column vector)
    # p.T = transpose of p = transpose of matrix with mean returns = 1 * N row vector 
    # w = weight vector of portfolio = N*1 weight column
    # p.T mulitpplied by w = scalar result 
    # IMPORTANT: PYTHON HAS REVERSED DEFINITION OF ROWS AND COLUMNS
    # ACCURATE NUMPY VERSION OF THE PREVIOUS EQUATION WOULD BE R = W * P.T
    
    mu = w * p.T
    
    # C is covariance of returns = N * N MATRIX =
    # NOTE: if use std(array(ret_vec).T*w) we would get a slightly different ’bullet’. NO COVARIANCE 
    # In the covariance matrix, the values of the diagonal represent the simple variances of each asset 
    # while the off-diagonals are the variances between the assets. 
    # By using ordinary std() we effectively only regard the diagonal and miss the rest.
    sigma = np.sqrt(w * C * w.T)
    
    # This recursion reduces outliers to keep plots pretty
    if sigma > 2:
        return random_portfolio(returns)
    return mu, sigma




# Generate the mean returns and volatility for 500 random portfolios
# np.column_stack stacks 1-D arrays as columns into a 2-D array.

n_portfolios = 5000
means, stds = np.column_stack([ 
    random_portfolio(return_vec) 
    for i in range(n_portfolios)
    # if big range, use xrange in PYTHON 2
    # if need to iterate list many times, use range 
])

# Form a characteristic parabolic shape called the ‘Markowitz bullet‘ with the boundaries being called 
# the ‘efficient frontier‘, where we have the lowest variance for a given expected.
#'o' is circle marker
#plt.plot(stds, means, 'o', markersize=5)
#plt.xlabel('std')
#plt.ylabel('mean')
#plt.title('Mean and standard deviation of returns of randomly generated portfolios')



def efficient_frontier(returns):
    n = len(returns)
    returns = np.asmatrix(returns)
    
    
    # The mus vector produces a series of expected return values in a non-linear and more appropriate way.
    N = 100
    mus = [10**(5.0 * t/N - 1.0) for t in range(N)]
    
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(returns))
    pbar = opt.matrix(np.mean(returns, axis=1))
    
    # Create constraint matrices
    # np.eye --> Return a 2-D array with ones on the diagonal and zeros elsewhere.
    G = -opt.matrix(np.eye(n))   # negative n x n identity matrix
    h = opt.matrix(0.0, (n ,1)) # create a matrix with all zeros of size (n*1)
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)

    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x']
                  for mu in mus]
    
    # CALCULATE RISKS AND RETURNS FOR FRONTIER
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]
    
    # POLYFIT --> CALCULATE THE 2ND DEGREE POLYNOMIAL OF THE FRONTIER CURVE
    # Least squares polynomial fit --> fit a polynomial with degree deg to points (x,y)
    # Returns a vector of coefficients p that minimises the squared error.
    m1 = np.polyfit(returns, risks, 2) 
    x1 = np.sqrt(m1[2] / m1[0])
    
    # CALCULATE THE OPTIMAL PORTFOLIO
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    return np.asarray(wt), returns, risks



def plot_efficient_frontier(return_vec):
    # Yellow is the optimal portfolios for each of the desired returns (i.e. the mus). 
    weights, returns, risks = efficient_frontier(return_vec)
    plt.plot(stds, means, 'o')
    plt.ylabel('mean')
    plt.xlabel('std')
    plt.plot(risks, returns, 'y-o')
    plt.title('Efficient Frontier')
    
    #In addition, we get the one optimal portfolio returned:
    print(weights)
    
plot_efficient_frontier(return_vec)

TypeError: unsupported operand type(s) for /: 'str' and 'str'